In [1]:
import base64
import json
import os
import requests
import pandas as pd
from prophet import Prophet
from tqdm import tqdm
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from utils import upload
import dropbox
import time
import datetime
import math

In [2]:
allData = pd.read_csv(
    'https://www.dropbox.com/scl/fi/ksf0nbmmiort5khbrgr61/allData.csv?rlkey=75e735fjk4ifttjt553ukxt3k&dl=1')
allData.ds = pd.to_datetime(allData.ds)

df = allData.copy()
df.ds = pd.to_datetime(df.ds)

df['total_tbs'] = df[['TRG_HALLWAY_TBS',
                      'POD_GREEN_TBS',
                      'POD_YELLOW_TBS',
                      'POD_ORANGE_TBS',
                      'RAZ_TBS',
                      'AMBVERTTBS',
                      'QTrack_TBS',
                      'Garage_TBS']].sum(axis=1)
df['vert_tbs'] = df[[
    'RAZ_TBS',
    'AMBVERTTBS',
    'QTrack_TBS',
    'Garage_TBS']].sum(axis=1)
df['pod_tbs'] = df[['TRG_HALLWAY_TBS',
                    'POD_GREEN_TBS',
                    'POD_YELLOW_TBS',
                    'POD_ORANGE_TBS',
                    ]].sum(axis=1)
df.tail()

tbs_columns = ['total_tbs', 'vert_tbs', 'pod_tbs']

output = pd.DataFrame()
FIRST_RUN = True

for column in tqdm(tbs_columns):
    if column in ['ds']:
        continue
    try:
        print('working on '+column)

        m = Prophet(interval_width=0.95)
        m.fit(df[['ds', column]].rename(columns={column: 'y'}))
        future = m.make_future_dataframe(periods=24*1, freq='h')
        # forecast = m.predict(future.tail(24*14))
        forecast = m.predict(future)
        if FIRST_RUN:
            output['ds'] = forecast['ds']

        for forecast_column in ['yhat', 'yhat_lower', 'yhat_upper']:
            kwargs = {column+'_'+forecast_column: forecast[forecast_column]}
            output = output.assign(**kwargs)
        FIRST_RUN = False
    except:
        print(column + ' failed')

output

  0%|          | 0/3 [00:00<?, ?it/s]

working on total_tbs


21:28:07 - cmdstanpy - INFO - Chain [1] start processing
21:28:44 - cmdstanpy - INFO - Chain [1] done processing
 33%|███▎      | 1/3 [00:47<01:35, 47.51s/it]

working on vert_tbs


21:28:54 - cmdstanpy - INFO - Chain [1] start processing
21:29:23 - cmdstanpy - INFO - Chain [1] done processing
 67%|██████▋   | 2/3 [01:25<00:42, 42.11s/it]

working on pod_tbs


21:29:33 - cmdstanpy - INFO - Chain [1] start processing
21:29:43 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 3/3 [01:45<00:00, 35.15s/it]


,ds,total_tbs_yhat,total_tbs_yhat_lower,total_tbs_yhat_upper,vert_tbs_yhat,vert_tbs_yhat_lower,vert_tbs_yhat_upper,pod_tbs_yhat,pod_tbs_yhat_lower,pod_tbs_yhat_upper
0,2021-01-01 01:00:00,2.856202,-13.380996,19.953043,1.080689,-12.995278,16.006731,1.832842,-3.132387,6.693993
1,2021-01-01 02:00:00,1.811500,-14.669720,18.204236,0.742780,-13.510002,15.507534,1.125800,-4.032666,6.215540
2,2021-01-01 03:00:00,-0.539329,-16.469852,15.580435,-0.984073,-15.715458,13.989769,0.501574,-4.411373,5.406146
3,2021-01-01 04:00:00,-3.695369,-19.464995,12.884029,-3.693186,-18.748017,11.141964,0.054402,-5.237065,5.245651
4,2021-01-01 05:00:00,-6.331978,-22.736101,10.820018,-6.090968,-20.053432,8.590929,-0.184648,-5.296954,4.827438
...,...,...,...,...,...,...,...,...,...,...
42513,2025-11-10 17:00:00,34.455642,17.798174,51.322668,27.677970,13.111429,42.647904,6.742026,2.101290,11.628200
42514,2025-11-10 18:00:00,32.121721,14.298359,49.758177,25.775958,11.147204,39.885130,6.310126,1.429752,11.359592
42515,2025-11-10 19:00:00,30.005727,12.941950,45.946460,24.018317,8.946206,39.289960,5.951769,0.685555,10.802216
42516,2025-11-10 20:00:00,28.305913,12.058935,44.730403,22.502924,7.935569,37.079321,5.767358,1.102409,10.658388


In [5]:
output.tail(24)

,ds,total_tbs_yhat,total_tbs_yhat_lower,total_tbs_yhat_upper,vert_tbs_yhat,vert_tbs_yhat_lower,vert_tbs_yhat_upper,pod_tbs_yhat,pod_tbs_yhat_lower,pod_tbs_yhat_upper
42494,2025-11-09 22:00:00,20.675281,4.206508,37.292741,16.151971,2.862440,30.148986,4.487004,-0.150238,10.015694
42495,2025-11-09 23:00:00,19.995447,3.925151,35.752768,15.660284,1.795214,29.715568,4.298883,-0.404011,9.227508
42496,2025-11-10 00:00:00,19.877016,3.930949,35.882865,15.948972,0.287032,30.352157,3.891774,-1.034308,8.735718
42497,2025-11-10 01:00:00,19.829776,3.832604,36.317775,16.483732,1.945740,31.350312,3.309767,-1.366218,8.147255
42498,2025-11-10 02:00:00,18.975387,2.207701,34.941121,16.264000,1.821720,30.943538,2.675095,-2.092197,7.564169
42499,2025-11-10 03:00:00,16.808120,0.966377,33.289208,14.648397,0.457576,28.005928,2.123416,-2.819490,6.648946
42500,2025-11-10 04:00:00,13.827259,-2.473615,29.883579,12.042458,-1.872726,27.317021,1.748481,-3.015086,6.569991
42501,2025-11-10 05:00:00,11.355987,-4.502713,27.091794,9.738783,-4.853913,24.873755,1.580892,-3.742005,6.358743
42502,2025-11-10 06:00:00,10.601409,-5.569959,27.348745,8.963779,-7.202890,23.625218,1.601366,-3.345766,6.567701
42503,2025-11-10 07:00:00,11.763688,-3.441010,27.213195,9.954247,-3.928156,25.162332,1.773273,-3.113651,6.943351


In [2]:
allData = pd.read_csv(
    'https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&dl=1')

allData.ds = pd.to_datetime(allData.ds)
print(allData.tail(1))
print('length of allData: '+str(len(allData)))

df = allData.copy()
df.isna().sum().sum()
df.ds = pd.to_datetime(df.ds)

output = pd.DataFrame()
FIRST_RUN = True

tbs_columns = ['total_tbs', 'vert_tbs', 'pod_tbs']

# for column in tqdm(df.columns.to_list()):
for column in tqdm(tbs_columns):

    if column in ['ds']:
        continue
    try:
        print('working on '+column)

        m = Prophet(interval_width=0.95)
        m.fit(df[['ds', column]].rename(columns={column: 'y'}))
        future = m.make_future_dataframe(periods=24*1, freq='h')
        forecast = m.predict(future.tail(24*14))
        if FIRST_RUN:
            output['ds'] = forecast['ds']

        for forecast_column in ['yhat', 'yhat_lower', 'yhat_upper']:
            kwargs = {column+'_'+forecast_column: forecast[forecast_column]}
            output = output.assign(**kwargs)
        FIRST_RUN = False
    except:
        print(column + ' failed')

output.tail(24)

                       ds  INFLOW_STRETCHER  Infl_Stretcher_cum  \
42493 2025-11-09 21:00:00                 4                  98   

       INFLOW_AMBULATORY  Infl_Ambulatory_cum  Inflow_Total  Inflow_Cum_Total  \
42493                  2                   78             6               176   

       INFLOW_AMBULANCES  Infl_Ambulances_cum  FLS  ...  RAZ_CONS_MORE2H  \
42493                  3                   31    1  ...                3   

       RAZ_IMCONS_MORE4H  RAZ_XRAY_MORE2H  RAZ_CT_MORE2H1  PSYCH1  \
42493                  1                2               1      13   

       PSYCH_WAITINGADM  total_tbs  vert_tbs  pod_tbs  overflow  
42493                11         10         9        1         5  

[1 rows x 50 columns]
length of allData: 42494


  0%|          | 0/3 [00:00<?, ?it/s]

working on total_tbs


21:36:30 - cmdstanpy - INFO - Chain [1] start processing
21:37:09 - cmdstanpy - INFO - Chain [1] done processing
/Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/prophet/forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
 33%|███▎      | 1/3 [00:42<01:24, 42.04s/it]

working on vert_tbs


21:37:12 - cmdstanpy - INFO - Chain [1] start processing
21:37:39 - cmdstanpy - INFO - Chain [1] done processing
/Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/prophet/forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
 67%|██████▋   | 2/3 [01:12<00:34, 34.97s/it]

working on pod_tbs


21:37:42 - cmdstanpy - INFO - Chain [1] start processing
21:37:52 - cmdstanpy - INFO - Chain [1] done processing
/Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/prophet/forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(
100%|██████████| 3/3 [01:25<00:00, 28.40s/it]


In [3]:
output.tail(24)

,ds,total_tbs_yhat,total_tbs_yhat_lower,total_tbs_yhat_upper,vert_tbs_yhat,vert_tbs_yhat_lower,vert_tbs_yhat_upper,pod_tbs_yhat,pod_tbs_yhat_lower,pod_tbs_yhat_upper
312,2025-11-09 22:00:00,20.675281,5.694935,36.741202,16.151971,2.196920,30.547510,4.487004,-0.377956,9.514013
313,2025-11-09 23:00:00,19.995447,3.517759,36.092591,15.660284,0.699998,29.884874,4.298883,-1.182601,9.082616
314,2025-11-10 00:00:00,19.877016,4.191148,36.838838,15.948972,1.490301,30.340017,3.891774,-1.248739,8.820259
315,2025-11-10 01:00:00,19.829776,3.082774,36.772520,16.483732,2.968293,31.706076,3.309767,-2.008425,8.610816
316,2025-11-10 02:00:00,18.975387,3.065281,35.407836,16.264000,2.058424,31.611898,2.675095,-2.052674,7.573164
317,2025-11-10 03:00:00,16.808120,0.306663,33.755919,14.648397,-0.385319,30.103047,2.123416,-2.915683,7.151386
318,2025-11-10 04:00:00,13.827259,-2.432392,29.860668,12.042458,-2.448142,26.352533,1.748481,-3.199183,6.325131
319,2025-11-10 05:00:00,11.355987,-5.447603,27.363160,9.738783,-4.945499,24.478209,1.580892,-3.482050,6.786753
320,2025-11-10 06:00:00,10.601409,-5.522095,26.876469,8.963779,-5.754066,24.080941,1.601366,-3.270475,6.615456
321,2025-11-10 07:00:00,11.763688,-6.127208,27.901022,9.954247,-4.850351,24.292572,1.773273,-3.367217,6.578214


In [6]:
import pytz

montreal_tz = pytz.timezone('America/Montreal')
current_hour = pd.Timestamp.now(tz=montreal_tz).floor('H')
timestamp_str = current_hour.strftime('%Y-%m-%d-%H')
timestamp_str

/var/folders/fy/5x01qzld283cfjx15gg5tsp40000gn/T/ipykernel_19646/2377643045.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  current_hour = pd.Timestamp.now(tz=montreal_tz).floor('H')


'2025-11-09-22'

In [19]:
def reformat_forecast(forecast):
    forecast_data = []

    for i in range(24):
        step_col = f"step{i}"
        quantile_20_col = f"step{i} 20.0%"  # Column name for 20th percentile
        quantile_80_col = f"step{i} 80.0%"  # Column name for 80th percentile

        forecast_data.append({
            "ds": forecast["ds"].iloc[-1] + pd.Timedelta(hours=i),
            "yhat": forecast[step_col].iloc[-1],
            # Add lower bound (20th percentile)
            "yhat_lower": forecast[quantile_20_col].iloc[-1],
            # Add upper bound (80th percentile)
            "yhat_upper": forecast[quantile_80_col].iloc[-1]
        })

    new_forecast_df = pd.DataFrame(forecast_data)
    return new_forecast_df

In [17]:
import os, tempfile, requests
from neuralprophet import load
import pandas as pd

def fetch_model_to_tmp(url: str, token: str | None = None) -> str:
    headers = {}
    if token:
        headers["Authorization"] = f"token {token}"
    with requests.get(url, headers=headers, stream=True, timeout=120) as r:
        r.raise_for_status()
        fd, tmp_path = tempfile.mkstemp(suffix=".np")
        with os.fdopen(fd, "wb") as f:
            for chunk in r.iter_content(chunk_size=1<<20):
                if chunk:
                    f.write(chunk)
    return tmp_path

MODEL_VERSION = os.getenv("MODEL_VERSION", "1")
ASSET = f"inflow_total_np-{MODEL_VERSION}.np"
URL = f"https://github.com/drdevinhopkins/ed-flow-2023/releases/download/v{MODEL_VERSION}/{ASSET}"
TOKEN = os.getenv("GITHUB_TOKEN")  # required if private

local_path = fetch_model_to_tmp(URL, TOKEN)
inflow_total_np_m = load(local_path)

MODEL_VERSION = os.getenv("MODEL_VERSION", "1")
ASSET = f"total_tbs-{MODEL_VERSION}.np"
URL = f"https://github.com/drdevinhopkins/ed-flow-2023/releases/download/v{MODEL_VERSION}/{ASSET}"
TOKEN = os.getenv("GITHUB_TOKEN")  # required if private

local_path = fetch_model_to_tmp(URL, TOKEN)
total_tbs_m = load(local_path)
# ... make_future_dataframe + predict as usual

In [20]:
data = pd.read_csv(
    'https://www.dropbox.com/scl/fi/ksf0nbmmiort5khbrgr61/allData.csv?rlkey=75e735fjk4ifttjt553ukxt3k&raw=1')
data.ds = pd.to_datetime(data.ds)
data = data.sort_values('ds')

df = data.copy()
df = df[['ds', 'Inflow_Total']].rename(
    columns={'ds': 'ds', 'Inflow_Total': 'y'})
df.tail()
df_future = inflow_total_np_m.make_future_dataframe(df[['ds', 'y']], periods=24)

if hasattr(inflow_total_np_m, "trainer") and inflow_total_np_m.trainer is not None:
    # Disable the built-in TensorBoard logger
    inflow_total_np_m.trainer.logger = False
    # Also disable callbacks that might try to log
    inflow_total_np_m.trainer.callbacks = []

forecast = inflow_total_np_m.predict(df_future, decompose=False, raw=True)
output_df = reformat_forecast(forecast)
inflow_total_np = output_df.copy()
inflow_total_np

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1120: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.887% of the data.
WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1120: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1120: FutureWarning: Series.view is deprecated and w

,ds,yhat,yhat_lower,yhat_upper
0,2025-11-09 23:00:00,7.420492,5.767215,9.109963
1,2025-11-10 00:00:00,7.050033,5.575466,8.705591
2,2025-11-10 01:00:00,7.414898,5.958930,8.946630
3,2025-11-10 02:00:00,6.913895,5.293740,8.768136
4,2025-11-10 03:00:00,6.454420,5.013968,8.013824
5,2025-11-10 04:00:00,5.511346,4.231073,6.920774
6,2025-11-10 05:00:00,4.941678,3.631611,6.235168
7,2025-11-10 06:00:00,4.335242,3.345657,5.344738
8,2025-11-10 07:00:00,4.306570,3.280541,5.342027
9,2025-11-10 08:00:00,3.893933,3.162746,4.612926


In [21]:
data['y'] = data['POD_GREEN_TBS']+data['POD_YELLOW_TBS']+data['POD_ORANGE_TBS'] + \
    data['TRG_HALLWAY_TBS']+data['RAZ_TBS'] + \
    data['AMBVERTTBS']+data['QTrack_TBS']+data['Garage_TBS']
df = data.copy()
df = df[['ds', 'y', 'Inflow_Total']]

# params = {
#     'growth': 'off',
#     'yearly_seasonality': True,
#     'weekly_seasonality': True,
#     'daily_seasonality': True,
#     'n_lags': 48,
#     'n_forecasts': 24,
#     'epochs': 50,
#     'quantiles': [0.2, 0.5, 0.8]
# }

# m = NeuralProphet(**params)
# m.set_plotting_backend("plotly-static")
# m.add_future_regressor('Inflow_Total')
# metrics = m.fit(df, freq='h', progress='plot')

df_future = total_tbs_m.make_future_dataframe(df[['ds', 'y', 'Inflow_Total']], periods=24, regressors_df=inflow_total_np[[
                                    'ds', 'yhat']].rename(columns={'ds': 'ds', 'yhat': 'Inflow_Total'}))

if hasattr(total_tbs_m, "trainer") and total_tbs_m.trainer is not None:
    # Disable the built-in TensorBoard logger
    total_tbs_m.trainer.logger = False
    # Also disable callbacks that might try to log
    total_tbs_m.trainer.callbacks = []

forecast = total_tbs_m.predict(df_future, decompose=False, raw=True)

output_df = reformat_forecast(forecast)

output_df

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1120: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency h corresponds to 99.887% of the data.
WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1120: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/neuralprophet/df_utils.py:1120: FutureWarning: Series.view is deprecated and w

,ds,yhat,yhat_lower,yhat_upper
0,2025-11-09 23:00:00,5.004720,-0.197128,9.533113
1,2025-11-10 00:00:00,6.595116,1.732529,10.937476
2,2025-11-10 01:00:00,7.137352,2.206991,11.219727
3,2025-11-10 02:00:00,7.724275,3.209178,11.621021
4,2025-11-10 03:00:00,7.093514,2.900593,10.884163
5,2025-11-10 04:00:00,7.287995,3.210037,10.931965
6,2025-11-10 05:00:00,9.214114,5.665488,12.629950
7,2025-11-10 06:00:00,11.407402,8.159098,14.655358
8,2025-11-10 07:00:00,9.819212,6.634755,12.888511
9,2025-11-10 08:00:00,11.882502,8.412668,15.304207


In [ ]:
output_df.to_csv('total_tbs_np.csv', index=False)

dropbox_app_key = os.environ.get("DROPBOX_APP_KEY")
dropbox_app_secret = os.environ.get("DROPBOX_APP_SECRET")
dropbox_refresh_token = os.environ.get("DROPBOX_REFRESH_TOKEN")

# exchange the authorization code for an access token:
token_url = "https://api.dropboxapi.com/oauth2/token"
params = {
    "grant_type": "refresh_token",
    "refresh_token": dropbox_refresh_token,
    "client_id": dropbox_app_key,
    "client_secret": dropbox_app_secret
}
r = requests.post(token_url, data=params)
# print(r.text)

dropbox_access_token = r.json()['access_token']

dbx = dropbox.Dropbox(dropbox_access_token)

upload(dbx, 'total_tbs_np.csv', '', '',
            'total_tbs_np.csv', overwrite=True)

print(output_df)